In [7]:
import os
import asyncio
from sqlalchemy import create_engine
from typing import Any
import asyncpg
from dotenv import load_dotenv, find_dotenv
from langchain_community.utilities.sql_database import SQLDatabase


In [8]:
_ = load_dotenv(find_dotenv())
postgres_uri = os.environ.get("DATABASE")

In [9]:
import pandas as pd

In [10]:
class PostgreSQL:
    def __init__(self,
                 *,
                 uri: Any | None=None,
                 **kwargs,
                 )-> None:
        self._dialect = "postgresql"
        self.uri = uri
        self.config = kwargs
        self.conn = None

    @property
    def dialect(self, ):
        return self._dialect

    async def connect(self, ) -> None:
        self.conn = await asyncpg.connect(
            dsn=self.uri,
            **self.config,                                          
        )

    async def create_table_schema(self, table_name: str) -> str:
        # Get columns
        columns = await self.conn.fetch(f"""
            SELECT column_name, data_type, is_nullable, character_maximum_length
            FROM information_schema.columns
            WHERE table_name = '{table_name}'
            ORDER BY ordinal_position;
        """)

        # Get primary keys
        pk_rows = await self.conn.fetch(f"""
            SELECT a.attname
            FROM pg_index i
            JOIN pg_attribute a ON a.attrelid = i.indrelid AND a.attnum = ANY(i.indkey)
            WHERE i.indrelid = '{table_name}'::regclass AND i.indisprimary;
        """)
        primary_keys = [row['attname'] for row in pk_rows]

        # Get foreign keys
        fk_rows = await self.conn.fetch(f"""
            SELECT
                kcu.column_name,
                ccu.table_name AS foreign_table,
                ccu.column_name AS foreign_column
            FROM information_schema.table_constraints AS tc
            JOIN information_schema.key_column_usage AS kcu
                ON tc.constraint_name = kcu.constraint_name
            JOIN information_schema.constraint_column_usage AS ccu
                ON ccu.constraint_name = tc.constraint_name
            WHERE tc.constraint_type = 'FOREIGN KEY' AND tc.table_name = '{table_name}';
        """)

        # Build column definitions
        col_defs = []
        for col in columns:
            name = f'"{col["column_name"]}"'
            dtype = col["data_type"].upper()
            if dtype == "CHARACTER VARYING":
                dtype = f'NVARCHAR({col["character_maximum_length"]})'
            elif dtype == "CHARACTER":
                dtype = f'NCHAR({col["character_maximum_length"]})'
            elif dtype == "TEXT":
                dtype = "TEXT"
            elif dtype == "INTEGER":
                dtype = "INTEGER"
            # Add more type mappings if needed

            nullable = "NOT NULL" if col["is_nullable"] == "NO" else ""
            col_defs.append(f"{name} {dtype} {nullable}".strip())

        # Add primary key
        if primary_keys:
            pk = ', '.join(f'"{key}"' for key in primary_keys)
            col_defs.append(f"PRIMARY KEY ({pk})")

        # Add foreign keys
        for fk in fk_rows:
            col_defs.append(
                f'FOREIGN KEY("{fk["column_name"]}") REFERENCES "{fk["foreign_table"]}" ("{fk["foreign_column"]}")'
            )

        # Final SQL
        sql = f'CREATE TABLE "{table_name}" (\n\t' + ',\n\t'.join(col_defs) + '\n);'     
        return sql
    
    
    async def list_tables(self, )-> list[str]:
        rows = await self.conn.fetch("""
            SELECT tablename
            FROM pg_catalog.pg_tables
            WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema';
        """)
        return [row['tablename'].title() for row in rows]

    async def get_example(self, table_name: str) -> str:
         # Execute a query
        values = await self.conn.fetch(
            f"SELECT * FROM {table_name} LIMIT 3;",

        )
        examples = []
        for item in values:
            examples.append(list(item.items()))

        records = [dict(row) for row in examples]
        
        table = pd.DataFrame(records).to_string()

        return ("/*\n" + f"3 rows from '{table_name}' table:\n" + table + "\n"+
                "*/"
        )

    async def get_all_table_schema_example(self) -> str:
        table_names = await self.list_tables()
        all_schema = ""
        for t_names in table_names:
            tab = await self.create_table_schema(t_names.lower())
            example = await self.get_example(t_names.lower())

            all_schema += tab + "\n\n" + example + "\n\n"
            
        return all_schema

    async def execute(self, query: str) -> asyncpg.Record:
        # Execute a query
        values = await self.conn.fetch(
            query,

        )
        await self.clean_up()
        return values
    
    async def clean_up(self) -> None:
        await self.conn.close()
        
psql = PostgreSQL(uri=postgres_uri)
await psql.connect()


In [11]:
await psql.list_tables()


['Artist',
 'Album',
 'Employee',
 'Customer',
 'Invoice',
 'Invoice_Line',
 'Track',
 'Playlist',
 'Playlist_Track',
 'Genre',
 'Media_Type']

In [12]:
res = await psql.get_example("customer")
print(res)

/*
3 rows from 'customer' table:
   customer_id first_name  last_name                                           company                          address                 city state  country postal_code               phone                 fax                  email  support_rep_id
0            1       Luís  Gonçalves  Embraer - Empresa Brasileira de Aeronáutica S.A.  Av. Brigadeiro Faria Lima, 2170  São José dos Campos    SP   Brazil   12227-000  +55 (12) 3923-5555  +55 (12) 3923-5566   luisg@embraer.com.br               3
1            2     Leonie     Köhler                                              None          Theodor-Heuss-Straße 34            Stuttgart  None  Germany       70174    +49 0711 2842222                None  leonekohler@surfeu.de               5
2            3   François   Tremblay                                              None                1498 rue Bélanger             Montréal    QC   Canada     H2G 1A7   +1 (514) 721-4711                None    ftremblay@gmail

In [13]:
print(psql.dialect)


postgresql


In [14]:
schema = await psql.create_table_schema("album")
print(schema)

CREATE TABLE "album" (
	"album_id" INTEGER NOT NULL,
	"title" NVARCHAR(160) NOT NULL,
	"artist_id" INTEGER NOT NULL,
	PRIMARY KEY ("album_id"),
	FOREIGN KEY("artist_id") REFERENCES "artist" ("artist_id")
);


In [15]:
schemas_examples = await psql.get_all_table_schema_example()
print(schemas_examples)

CREATE TABLE "artist" (
	"artist_id" INTEGER NOT NULL,
	"name" NVARCHAR(120),
	PRIMARY KEY ("artist_id")
);

/*
3 rows from 'artist' table:
   artist_id       name
0          1      AC/DC
1          2     Accept
2          3  Aerosmith
*/

CREATE TABLE "album" (
	"album_id" INTEGER NOT NULL,
	"title" NVARCHAR(160) NOT NULL,
	"artist_id" INTEGER NOT NULL,
	PRIMARY KEY ("album_id"),
	FOREIGN KEY("artist_id") REFERENCES "artist" ("artist_id")
);

/*
3 rows from 'album' table:
   album_id                                  title  artist_id
0         1  For Those About To Rock We Salute You          1
1         2                      Balls to the Wall          2
2         3                      Restless and Wild          2
*/

CREATE TABLE "employee" (
	"employee_id" INTEGER NOT NULL,
	"last_name" NVARCHAR(20) NOT NULL,
	"first_name" NVARCHAR(20) NOT NULL,
	"title" NVARCHAR(30),
	"reports_to" INTEGER,
	"birth_date" TIMESTAMP WITHOUT TIME ZONE,
	"hire_date" TIMESTAMP WITHOUT TIME ZONE,
	"addre

In [43]:
from langchain.chat_models import init_chat_model

model = "gpt-4o"
model_provider = "openai"
# Instantiate the chat model
llm = init_chat_model(f"{model_provider}:{model}")

In [12]:
# test that it works
output = await llm.ainvoke("hi!") # call the model
output.content  # Get the content

'Hello! How can I assist you today?'

In [17]:
async for step in llm.astream("hello how are you"):
    print(step.content, end=" ")

 Hello !  I'm  just  a  computer  program ,  so  I  don't  have  feelings ,  but  I'm  here  and  ready  to  help  you .  How  can  I  assist  you  today ?  

In [1]:
from business_copilot.biz_analytics.db_tools import (list_tables, 
                                                     get_relevant_schema_example, 
                                                     resolve_error, 
                                                     execute_query,
                                                     double_check_query,
                                                     clean_up)
from business_copilot.biz_analytics.prompts import POSTGRES_SYSTEM_MESSAGE
from langchain_core.prompts import PromptTemplate
from langgraph.prebuilt import create_react_agent

In [2]:
import pprint
pprint.pprint(POSTGRES_SYSTEM_MESSAGE)

('\n'
 'You are an agent designed to interact with a POSTGRESQL database.\n'
 'Given an input question, create a syntactically correct query to run,\n'
 'then look at the results of the query and return the answer. Unless the '
 'user\n'
 'specifies a specific number of examples they wish to obtain, always limit '
 'your\n'
 'query to at most 5 results.\n'
 '\n'
 'To start you should ALWAYS look at the list of tables in the database to to '
 'be sure your predicted table exist \n'
 'so you can query. Do NOT skip this step.\n'
 '\n'
 'check the list of tables again to be sure they are all relevant before\n'
 'YOU get example and schema for each of the table you predicted and that '
 'exist. \n'
 'to have a context you can work with.\n'
 'ONLY Then you execute the query with the most relevant table.\n'
 '\n'
 'You can order the results by a relevant column to return the most '
 'interesting\n'
 'examples in the database. Never query for all the columns from a specific '
 'table,\n'
 'onl

In [2]:
# Get the tools
tools = [
    list_tables, get_relevant_schema_example,
    resolve_error, execute_query, 
    double_check_query, clean_up
]

In [10]:
from langchain.chat_models import init_chat_model
# Create graph
model = "gpt-4o"
llm = init_chat_model(f"openai:{model}", temperature=0.0)
#agent_executor = create_react_agent(llm , tools=tools, prompt=POSTGRES_SYSTEM_MESSAGE)

In [5]:
question =  "Which country's customers spent the most?"
async for step in agent_executor.astream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Which country's customers spent the most?
================================== Ai Message ==================================
Tool Calls:
  list_tables (call_V9XuWjU3fvcti809ruQvDpE4)
 Call ID: call_V9XuWjU3fvcti809ruQvDpE4
  Args:
================================= Tool Message =================================
Name: list_tables

["artist", "album", "employee", "customer", "invoice", "invoice_line", "track", "playlist", "playlist_track", "genre", "media_type"]
================================== Ai Message ==================================
Tool Calls:
  get_relevant_schema_example (call_Z8Zu1gra6HcMW0DgqegJXEhV)
 Call ID: call_Z8Zu1gra6HcMW0DgqegJXEhV
  Args:
    table_names: ['customer', 'invoice']
================================= Tool Message =================================
Name: get_relevant_schema_example

CREATE TABLE "customer" (
	"customer_id" INTEGER NOT NULL,
	"first_name" NVARCHAR(40) NOT NULL,


### Memory

In [3]:


from langgraph.prebuilt.chat_agent_executor import AgentState
from langmem.short_term import SummarizationNode, RunningSummary
from langchain_core.messages.utils import count_tokens_approximately
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.checkpoint.memory import InMemorySaver
from typing import Any

In [5]:
from langchain.chat_models import init_chat_model
summarization_llm = init_chat_model("openai:gpt-4o", temperature=0.0)

In [6]:
summarization_node = SummarizationNode( 
    token_counter=count_tokens_approximately,
    model=summarization_llm,
    max_tokens=356,  # make it higher depends on question
    max_summary_tokens=256,
    output_messages_key="summarized_llm_messages",
)

In [7]:
class State(AgentState):
    # NOTE: We're adding this key to keep track of previous summary information
    # to make sure we're not summarizing on every LLM call
    context: dict[str, RunningSummary]

In [8]:
checkpointer = InMemorySaver()

In [11]:
agent_mem = create_react_agent(
    llm, tools=tools, 
    prompt=POSTGRES_SYSTEM_MESSAGE, 
    pre_model_hook=summarization_node,
    state_schema=State,
    checkpointer=checkpointer,
                              )

config = {
    "configurable": {
        "thread_id": "1"
    }
}



In [12]:
question =  "Which country's customers spent the most?"
async for step in agent_mem.astream(
    {"messages": [{"role": "user", "content": question}]},
    config=config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Which country's customers spent the most?
================================== Ai Message ==================================
Tool Calls:
  list_tables (call_Fg4TK5HhifrbgBFk0O1zo6Uk)
 Call ID: call_Fg4TK5HhifrbgBFk0O1zo6Uk
  Args:
================================= Tool Message =================================
Name: list_tables

["artist", "album", "employee", "customer", "invoice", "invoice_line", "track", "playlist", "playlist_track", "genre", "media_type"]
================================== Ai Message ==================================
Tool Calls:
  get_relevant_schema_example (call_W07OtF6mSE3XmX0VPGenah9K)
 Call ID: call_W07OtF6mSE3XmX0VPGenah9K
  Args:
    table_names: ['customer', 'invoice']
================================= Tool Message =================================
Name: get_relevant_schema_example

CREATE TABLE "customer" (
	"customer_id" INTEGER NOT NULL,
	"first_name" NVARCHAR(40) NOT NULL,


/home/nnaemeka/Desktop/business-copilot/.venv/lib/python3.13/site-packages/langmem/short_term/summarization.py:246: RuntimeWarning: Failed to trim messages to fit within max_tokens limit before summarization - falling back to the original message list. This may lead to exceeding the context window of the summarization LLM.
  warnings.warn(


================================= Tool Message =================================
Name: get_relevant_schema_example

CREATE TABLE "customer" (
	"customer_id" INTEGER NOT NULL,
	"first_name" NVARCHAR(40) NOT NULL,
	"last_name" NVARCHAR(20) NOT NULL,
	"company" NVARCHAR(80),
	"address" NVARCHAR(70),
	"city" NVARCHAR(40),
	"state" NVARCHAR(40),
	"country" NVARCHAR(40),
	"postal_code" NVARCHAR(10),
	"phone" NVARCHAR(24),
	"fax" NVARCHAR(24),
	"email" NVARCHAR(60) NOT NULL,
	"support_rep_id" INTEGER,
	PRIMARY KEY ("customer_id"),
	FOREIGN KEY("support_rep_id") REFERENCES "employee" ("employee_id")
);

/*
3 rows from 'customer' table:
   customer_id first_name  last_name                                           company                          address                 city state  country postal_code               phone                 fax                  email  support_rep_id
0            1       Luís  Gonçalves  Embraer - Empresa Brasileira de Aeronáutica S.A.  Av. Brigadeiro Faria Lima, 21

/home/nnaemeka/Desktop/business-copilot/.venv/lib/python3.13/site-packages/langmem/short_term/summarization.py:246: RuntimeWarning: Failed to trim messages to fit within max_tokens limit before summarization - falling back to the original message list. This may lead to exceeding the context window of the summarization LLM.
  warnings.warn(


================================= Tool Message =================================
Name: clean_up

null
================================== Ai Message ==================================

The country whose customers spent the most is the USA, with a total spending of 523.06.


In [13]:
question ="how much did USA spend again"
async for step in agent_mem.astream(
    {"messages": [{"role": "user", "content": question}]},
    config=config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

how much did USA spend again
================================ Human Message =================================

how much did USA spend again
================================== Ai Message ==================================

The customers from the USA spent a total of 523.06.


In [14]:
agent_mem.get_state(config)

StateSnapshot(values={'messages': [HumanMessage(content="Which country's customers spent the most?", additional_kwargs={}, response_metadata={}, id='a98975df-12d0-4239-972e-79110fc73cca'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Fg4TK5HhifrbgBFk0O1zo6Uk', 'function': {'arguments': '{}', 'name': 'list_tables'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 556, 'total_tokens': 566, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_80956533cb', 'id': 'chatcmpl-C5AkDBg9du39f0bnDlzhbcexYpq5a', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ddf9e917-7fa0-438c-86b6-b76aa78cb58a-0', tool_calls=[{'name': 'list_tables', 'args': {}, 'id': 'call_F